In [4]:
%pip install scikit-learn
%pip list
%pip install -r requirements.txt
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Package                Version
---------------------- ----------------
asttokens              2.4.1
certifi                2023.7.22
chardet                4.0.0
charset-normalizer     3.3.2
colorama               0.4.6
comm                   0.2.2
contourpy              1.2.1
cycler                 0.10.0
debugpy                1.8.1
decorator              5.1.1
distlib                0.3.8
dtaidistance           2.3.11
dtw-python             1.5.1
executing              2.0.1
fastdtw                0.3.4
filelock               3.13.4
fonttools              4.51.0
fsspec                 2024.3.1
fuzzywuzzy             0.18.0
idna                   2.10
imageio                2.34.0
imgaug                 0.4.0
intel-openmp           2021.4.0
ipykernel              6.29.4
ipython                8.25.0
jedi                   0.19.1
Jinja2                 3.1.3
joblib                 1.4.2
jupyter_client         8.6.2
jupyter_core           5.7.2
kiwisolver             1.4.5
lazy_loader 

ERROR: Could not find a version that satisfies the requirement torch==2.3.0+cu121 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0)
ERROR: No matching distribution found for torch==2.3.0+cu121

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import cv2
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor

In [ ]:
import numpy as np
import cv2
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from scipy.signal import convolve2d
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('best.pt')

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_uint8 = (image * 255).astype('uint8')  # Ensure image is uint8
    h = hog.compute(image_uint8)
    return h.flatten()

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    klt = cv2.goodFeaturesToTrack(gray_frame, maxCorners=100, qualityLevel=0.01, minDistance=10)
    klt = klt.reshape(-1, 2) if klt is not None else np.array([])
    return klt

def process_video(video_path, sample_rate=5):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        klt_features = extract_features(frame)
        if klt_features.size == 0:
            continue
        features.append(klt_features.mean(axis=0))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def synchronize_videos(video1_path, video2_path, sample_rate=5):
    print("Processing video 1...")
    features1 = process_video(video1_path, sample_rate)
    print("Processing video 2...")
    features2 = process_video(video2_path, sample_rate)
    
    if len(features1) == 0 or len(features2) == 0:
        print("Error: No features extracted from one or both videos")
        return 0, 0
    
    print("Computing SSMs...")
    ssm1 = compute_ssm(features1)
    ssm2 = compute_ssm(features2)
    
    print("Extracting SSM descriptors using HOG with log-polar...")
    descriptors1 = [extract_ssm_descriptor(ssm1, i) for i in range(len(features1))]
    descriptors2 = [extract_ssm_descriptor(ssm2, i) for i in range(len(features2))]
    
    if len(descriptors1) == 0 or len(descriptors2) == 0:
        print("Error: No valid descriptors extracted")
        return 0, 0
    
    print("Performing DTW with fastdtw...")
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print("Error: DTW returned empty path")
        return 0, 0
    
    time_shift_frames = int(np.median(shifts) * sample_rate)
    
    cap1 = cv2.VideoCapture(video1_path)
    fps = cap1.get(cv2.CAP_PROP_FPS)
    cap1.release()

    time_shift_seconds = time_shift_frames / fps

    return time_shift_frames, time_shift_seconds

# Example usage
video1_path = 'IC3-TGI-08nov-Hazel-Jennie-Spring.mp4'
video2_path = 'IC1-08nov-TGI-Hazel-Jennie-Spring.mp4'
 video3_path =   'IC5-TGI-08nov-Hazel-Jennie-Spring.mp4'
time_shift_frames, time_shift_seconds = synchronize_videos(video1_path, video2_path)
print(f"Estimated Time Shift: {time_shift_frames} frames")
print(f"Estimated Time Shift: {time_shift_seconds:.2f} seconds")


In [3]:
import numpy as np
import cv2
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor

def extract_features(frame):
    """
    Extract SIFT features from the frame.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors

def track_features(video_path, sample_rate):
    """
    Track features across frames to create trajectories.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Error opening video file: {video_path}")
    
    trajectories = []
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
    ret, old_frame = cap.read()
    if not ret:
        cap.release()
        raise ValueError(f"Error reading the first frame of the video file: {video_path}")
    
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=500, qualityLevel=0.01, minDistance=7, blockSize=7)
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            trajectories.append((a, b, c, d))
        
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
    
    cap.release()
    return trajectories

def filter_trajectories(trajectories):
    """
    Filter out noisy and non-informative trajectories.
    """
    filtered_trajectories = []
    for traj in trajectories:
        if np.linalg.norm(np.array(traj[:2]) - np.array(traj[2:])) > 2:  # Example threshold
            filtered_trajectories.append(traj)
    return filtered_trajectories

def find_potential_matches(trajectories1, trajectories2):
    """
    Find potential matches between trajectories from two videos.
    """
    matches = []
    for t1 in trajectories1:
        for t2 in trajectories2:
            if np.linalg.norm(np.array(t1[:2]) - np.array(t2[:2])) < 10:  # Example threshold
                matches.append((t1, t2))
    return matches

def ransac_optimization(matches):
    """
    Optimize synchronization parameters using RANSAC.
    """
    if len(matches) == 0:
        return []
    
    X = np.array([m[0][:2] for m in matches])
    y = np.array([m[1][:2] for m in matches])
    
    ransac = RANSACRegressor()
    ransac.fit(X, y)
    
    inliers = ransac.inlier_mask_
    return [matches[i] for i in range(len(matches)) if inliers[i]]

def synchronize_videos(video_paths, sample_rate=5):
    """
    Synchronize multiple videos by finding the time shifts between them.
    """
    trajectories = []
    for video_path in video_paths:
        try:
            traj = track_features(video_path, sample_rate)
            trajectories.append(traj)
        except ValueError as e:
            print(e)
            return None, None
    
    filtered_trajectories = [filter_trajectories(t) for t in trajectories]
    
    time_shift_frames = np.zeros((len(video_paths), len(video_paths)))
    time_shift_seconds = np.zeros((len(video_paths), len(video_paths)))
    
    for i in range(len(video_paths)):
        for j in range(i + 1, len(video_paths)):
            potential_matches = find_potential_matches(filtered_trajectories[i], filtered_trajectories[j])
            print(f"Potential matches between video {i} and video {j}: {len(potential_matches)}")
            refined_matches = ransac_optimization(potential_matches)
            print(f"Refined matches between video {i} and video {j}: {len(refined_matches)}")
            
            if refined_matches:
                features_i = [m[0] for m in refined_matches]
                features_j = [m[1] for m in refined_matches]
                
                distance, path = fastdtw(features_i, features_j, dist=euclidean)
                shift = [p1 - p2 for p1, p2 in path]
                shift = [s for s in shift if not np.isnan(s)]  # Remove NaN values
                print(f"Shifts between video {i} and video {j}: {shift}")
                if len(shift) > 0:
                    median_shift = int(np.median(shift) * sample_rate)
                    print(f"Median shift between video {i} and video {j}: {median_shift}")
                    time_shift_frames[i, j] = median_shift
                    time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds

# Example usage
video_paths = [
    'IC1-08nov-TGI-Hazel-Jennie-Spring.mp4',
    'IC3-TGI-08nov-Hazel-Jennie-Spring.mp4',
    'IC5-TGI-08nov-Hazel-Jennie-Spring.mp4'
]
time_shift_frames, time_shift_seconds = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

KeyboardInterrupt: 

In [11]:
import numpy as np
import cv2
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
import pandas as pd

def compute_ssm(features):
    """
    Compute the self-similarity matrix (SSM) for a set of features.
    """
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_hog_descriptor(image):
    """
    Extract Histogram of Oriented Gradients (HOG) descriptor from an image.
    """
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')  # Ensure image is uint8 and replace NaNs with 0
    h = hog.compute(image_uint8)
    return h.flatten()

def extract_log_polar_patch(ssm, i, window_size=10):
    """
    Extract a log-polar patch centered at (i, i) from the SSM.
    """
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    """
    Extract a descriptor from the SSM at index i.
    """
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def extract_features(frame):
    """
    Extract features from a frame using ORB feature detector.
    """
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)  # Increase number of features
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def process_video(video_path, sample_rate=5):
    """
    Process a video and extract features from sampled frames.
    """
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        features.append(orb_features.mean(axis=0))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def synchronize_videos(video_paths, sample_rate=5):
    """
    Synchronize multiple videos by finding the time shifts between them.
    """
    features_list = []
    for video_path in video_paths:
        print(f"Processing {video_path}...")
        features = process_video(video_path, sample_rate)
        if len(features) == 0:
            print(f"Error: No features extracted from {video_path}")
            return None, None
        features_list.append(features)
    
    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
            
            shifts = [p1 - p2 for p1, p2 in path]
            if len(shifts) == 0:
                print(f"Error: DTW returned empty path for video {i} and video {j}")
                return None, None
            
            # Apply RANSAC to filter noisy shifts
            shifts = np.array(shifts).reshape(-1, 1)
            indices = np.arange(len(shifts)).reshape(-1, 1)
            ransac = RANSACRegressor()
            ransac.fit(indices, shifts)
            inlier_mask = ransac.inlier_mask_
            
            median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

# Example usage
video_paths = [
'IC1-08nov-TGI-Hazel-Jennie-Spring.mp4',
    'IC3-TGI-08nov-Hazel-Jennie-Spring.mp4',
    'IC5-TGI-08nov-Hazel-Jennie-Spring.mp4'
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    # Display matrix with video names
    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)


Processing IC1-08nov-TGI-Hazel-Jennie-Spring.mp4...
Processed 100 frames
Processed 200 frames
Processed 300 frames
Processed 400 frames
Processed 500 frames
Processed 600 frames
Processed 700 frames
Processed 800 frames
Processed 900 frames
Processed 1000 frames
Processed 1100 frames
Processed 1200 frames
Processed 1300 frames
Processed 1400 frames
Processed 1500 frames
Processed 1600 frames
Processed 1700 frames
Processed 1800 frames
Processed 1900 frames
Processed 2000 frames
Processing IC3-TGI-08nov-Hazel-Jennie-Spring.mp4...
Processed 100 frames
Processed 200 frames
Processed 300 frames
Processed 400 frames
Processed 500 frames
Processed 600 frames
Processed 700 frames
Processed 800 frames
Processed 900 frames
Processed 1000 frames
Processed 1100 frames
Processed 1200 frames
Processed 1300 frames
Processed 1400 frames
Processed 1500 frames
Processed 1600 frames
Processed 1700 frames
Processed 1800 frames
Processed 1900 frames
Processed 2000 frames
Processed 2100 frames
Processed 22

In [12]:
import cv2
import numpy as np

def process_video_keypoints(video_path):
    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        # Read the next frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect keypoints and compute descriptors
        keypoints, descriptors = orb.detectAndCompute(gray_frame, None)

        # Draw keypoints on the frame
        frame_with_keypoints = cv2.drawKeypoints(frame, keypoints, None, color=(0, 255, 0), flags=0)
        
        # Display the frame with keypoints
        cv2.imshow('Keypoints', frame_with_keypoints)
        
        # Press 'q' to quit the video display
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the video capture object and close display window
    cap.release()
    cv2.destroyAllWindows()

# Example usage
video_path = 'IC1-08nov-TGI-Hazel-Jennie-Spring.mp4'
process_video_keypoints(video_path)

In [2]:
import numpy as np
import cv2
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
import pandas as pd

def compute_ssm(features):
    """
    Compute the self-similarity matrix (SSM) for a set of features.
    """
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_hog_descriptor(image):
    """
    Extract Histogram of Oriented Gradients (HOG) descriptor from an image.
    """
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')  # Ensure image is uint8 and replace NaNs with 0
    h = hog.compute(image_uint8)
    return h.flatten()

def extract_log_polar_patch(ssm, i, window_size=10):
    """
    Extract a log-polar patch centered at (i, i) from the SSM.
    """
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    """
    Extract a descriptor from the SSM at index i.
    """
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def extract_features(frame):
    """
    Extract features from a frame using SIFT feature detector.
    """
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def process_video(video_path, sample_rate=5):
    """
    Process a video and extract features from sampled frames.
    """
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        sift_features = extract_features(frame)
        if sift_features.size == 0:
            continue
        features.append(sift_features.mean(axis=0))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def synchronize_videos(video_paths, sample_rate=5):
    """
    Synchronize multiple videos by finding the time shifts between them.
    """
    features_list = []
    for video_path in video_paths:
        print(f"Processing {video_path}...")
        features = process_video(video_path, sample_rate)
        if len(features) == 0:
            print(f"Error: No features extracted from {video_path}")
            return None, None
        features_list.append(features)
    
    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
            
            shifts = [p1 - p2 for p1, p2 in path]
            if len(shifts) == 0:
                print(f"Error: DTW returned empty path for video {i} and video {j}")
                return None, None
            
            # Apply RANSAC to filter noisy shifts
            shifts = np.array(shifts).reshape(-1, 1)
            indices = np.arange(len(shifts)).reshape(-1, 1)
            ransac = RANSACRegressor()
            ransac.fit(indices, shifts)
            inlier_mask = ransac.inlier_mask_
            
            median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

# Example usage
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    # Display matrix with video names
    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)


Processing IC1-08nov-TGI-Hazel-Jennie-Spring.mp4...
Processed 100 frames
Processed 200 frames
Processed 300 frames
Processed 400 frames
Processed 500 frames
Processed 600 frames
Processed 700 frames
Processed 800 frames
Processed 900 frames
Processed 1000 frames


KeyboardInterrupt: 

In [4]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def compute_ssm(features):
    """
    Compute the self-similarity matrix (SSM) for a set of features.
    """
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_hog_descriptor(image):
    """
    Extract Histogram of Oriented Gradients (HOG) descriptor from an image.
    """
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')  # Ensure image is uint8 and replace NaNs with 0
    h = hog.compute(image_uint8)
    return h.flatten()

def extract_sift_descriptor(image):
    """
    Extract SIFT descriptor from an image.
    """
    gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_frame, None)
    return descriptors.flatten() if descriptors is not None else np.array([])

def extract_features(frame):
    """
    Extract features from a frame using SIFT and HOG.
    """
    sift_features = extract_sift_descriptor(frame)
    hog_features = extract_hog_descriptor(frame)
    combined_features = np.concatenate((sift_features, hog_features))
    return combined_features

def process_video(video_path, sample_rate=5):
    """
    Process a video and extract features from sampled frames.
    """
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        combined_features = extract_features(frame)
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=5):
    """
    Process multiple videos concurrently to extract features.
    """
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def synchronize_videos(video_paths, sample_rate=5):
    """
    Synchronize multiple videos by finding the time shifts between them.
    """
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            _, path = fastdtw(ssm1.flatten(), ssm2.flatten(), dist=euclidean)
            
            shifts = [p1 - p2 for p1, p2 in path]
            if len(shifts) == 0:
                print(f"Error: DTW returned empty path for video {i} and video {j}")
                return None, None
            
            # Apply RANSAC to filter noisy shifts
            shifts = np.array(shifts).reshape(-1, 1)
            indices = np.arange(len(shifts)).reshape(-1, 1)
            ransac = RANSACRegressor()
            ransac.fit(indices, shifts)
            inlier_mask = ransac.inlier_mask_
            
            median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

# Example usage
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    # Display matrix with video names
    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 132710400 bytes in function 'cv::OutOfMemoryError'


In [6]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

#keypoints and descriptors extraction
def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

#HOG descriptors extraction
def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

#SSM matrix computation
def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor
#
def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        features.append(orb_features.mean(axis=0))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)


def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list


def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
            
            shifts = [p1 - p2 for p1, p2 in path]
            if len(shifts) == 0:
                print(f"Error: DTW returned empty path for video {i} and video {j}")
                return None, None
            
            shifts = np.array(shifts).reshape(-1, 1)
            indices = np.arange(len(shifts)).reshape(-1, 1)
            ransac = RANSACRegressor()
            ransac.fit(indices, shifts)
            inlier_mask = ransac.inlier_mask_
            
            median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

# application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1200 frames
Processed 1300 frames
Processed 1300 frames
Processed 1300 frames
Processed 1400 frames
Processed 1400 frames
Processed 1500 frames
Processed 1400 frames
Processed 1500 frames
Processed 1600 frames
Processed 1500 frames
Processed 1600 

In [9]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        features.append(orb_features.mean(axis=0))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
            
            shifts = [p1 - p2 for p1, p2 in path]
            if len(shifts) == 0:
                print(f"Error: DTW returned empty path for video {i} and video {j}")
                return None, None
            
            shifts = np.array(shifts).reshape(-1, 1)
            indices = np.arange(len(shifts)).reshape(-1, 1)
            ransac = RANSACRegressor()
            ransac.fit(indices, shifts)
            inlier_mask = ransac.inlier_mask_
            
            median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

# application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1000 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1100 frames
Processed 1300 frames
Processed 1300 frames
Processed 1200 frames
Processed 1400 frames
Processed 1400 frames
Processed 1300 frames
Processed 1500 frames
Processed 1500 frames
Processed 1400 frames
Processed 1600 frames
Processed 1600 frames
Processed 1500 

C:\Users\orchi\AppData\Local\Temp\ipykernel_3460\2482400381.py:19: RuntimeWarning: invalid value encountered in cast
  image_uint8 = np.nan_to_num(image * 255).astype('uint8')


Performing DTW with fastdtw for video 0 and video 1...
Median shift between video 0 and video 1: -72 frames
Computing SSMs for video 0 and video 2...
Extracting SSM descriptors for video 0 and video 2...
Performing DTW with fastdtw for video 0 and video 2...
Median shift between video 0 and video 2: -250 frames
Computing SSMs for video 1 and video 2...
Extracting SSM descriptors for video 1 and video 2...
Performing DTW with fastdtw for video 1 and video 2...
Median shift between video 1 and video 2: -65 frames
Estimated Time Shifts (in frames):
[[   0.  -72. -250.]
 [  72.    0.  -65.]
 [ 250.   65.    0.]]
Estimated Time Shifts (in seconds):
[[ 0.         -2.4024024  -8.34167501]
 [ 2.4024024   0.         -2.1688355 ]
 [ 8.34167501  2.1688355   0.        ]]
Time Shifts (in frames) with video names:
                                       IC1-08nov-TGI-Hazel-Jennie-Spring.mp4  \
IC1-08nov-TGI-Hazel-Jennie-Spring.mp4                                    0.0   
IC3-TGI-08nov-Hazel-Jennie-S

In [4]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=900)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        features.append(orb_features.mean(axis=0))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
            
            shifts = [p1 - p2 for p1, p2 in path]
            if len(shifts) == 0:
                print(f"Error: DTW returned empty path for video {i} and video {j}")
                return None, None
            
            shifts = np.array(shifts).reshape(-1, 1)
            indices = np.arange(len(shifts)).reshape(-1, 1)
            ransac = RANSACRegressor()
            ransac.fit(indices, shifts)
            inlier_mask = ransac.inlier_mask_
            
            median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

# application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)

Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1200 frames
Processed 1300 frames
Processed 1300 frames
Processed 1300 frames
Processed 1400 frames
Processed 1400 frames
Processed 1400 frames
Processed 1500 frames
Processed 1500 frames
Processed 1500 frames
Processed 1600 frames
Processed 1600 

KeyboardInterrupt: 

In [1]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        sift_features = extract_features(frame)
        if sift_features.size == 0:
            continue
        features.append(sift_features.mean(axis=0))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
            
            shifts = [p1 - p2 for p1, p2 in path]
            if len(shifts) == 0:
                print(f"Error: DTW returned empty path for video {i} and video {j}")
                return None, None
            
            shifts = np.array(shifts).reshape(-1, 1)
            indices = np.arange(len(shifts)).reshape(-1, 1)
            ransac = RANSACRegressor()
            ransac.fit(indices, shifts)
            inlier_mask = ransac.inlier_mask_
            
            median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

def align_and_save_videos(video_paths, time_shift_frames, output_path):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width * len(video_paths), frame_height))
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            frames.append(frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
        
        cv2.imshow('Synchronized Videos', combined_frame)
        
        if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            break
    
    for cap in caps:
        cap.release()
    out.release()
    cv2.destroyAllWindows()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
output_path = r"synchronized_output.avi"
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_save_videos(video_paths, time_shift_frames, output_path)

Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1200 frames
Processed 1300 frames


In [ ]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1100)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        features.append(orb_features.mean(axis=0))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None
            
            print(f"Computing DTW shift for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

def align_and_display_videos(video_paths, time_shift_frames):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            frames.append(frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        cv2.imshow('Synchronized Videos', combined_frame)
        
        if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            break
    
    for cap in caps:
        cap.release()
    cv2.destroyAllWindows()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames)


In [15]:
import cv2
import numpy as np

def resize_frame(frame, scale=0.5):
    """
    Resize the frame to a smaller size for display.
    """
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    return cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)

def process_video_keypoints(video_path):
    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        # Read the next frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect keypoints and compute descriptors
        keypoints, descriptors = orb.detectAndCompute(gray_frame, None)

        # Draw keypoints on the frame
        frame_with_keypoints = cv2.drawKeypoints(frame, keypoints, None, color=(0, 255, 0), flags=0)
        
        # Resize the frame with keypoints
        frame_with_keypoints_resized = resize_frame(frame_with_keypoints, scale=0.5)
        
        # Display the frame with keypoints
        cv2.imshow('Keypoints', frame_with_keypoints_resized)
        
        # Press 'q' to quit the video display
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the video capture object and close display window
    cap.release()
    cv2.destroyAllWindows()

# Example usage
video_path = 'IC1-08nov-TGI-Hazel-Jennie-Spring.mp4'
process_video_keypoints(video_path)


In [14]:
#presenting here the matching



import cv2
import numpy as np
import pandas as pd

def resize_frame(frame, scale=0.5):
    """
    Resize the frame to a smaller size for display.
    """
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    return cv2.resize(frame, (width, height), interpolation=cv2.INTER_AREA)

def find_sync_point(video_paths):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    orb = cv2.ORB_create()
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    fps_list = [cap.get(cv2.CAP_PROP_FPS) for cap in caps]

    offsets = np.zeros((len(video_paths), len(video_paths)))

    while True:
        frames = []
        grays = []
        keypoints = []
        descriptors = []

        for idx, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                print(f"End of video {idx + 1}")
                break
            frames.append(frame)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            grays.append(gray)
            kp, des = orb.detectAndCompute(gray, None)
            keypoints.append(kp)
            descriptors.append(des)
            print(f"Video {idx + 1}: {len(kp)} keypoints detected")

        if len(frames) != len(video_paths):
            break  # End of one of the videos

        # Match descriptors between all pairs of videos
        matches = []
        for i in range(len(video_paths)):
            for j in range(i + 1, len(video_paths)):
                if descriptors[i] is not None and descriptors[j] is not None:
                    match = bf.match(descriptors[i], descriptors[j])
                    match = sorted(match, key=lambda x: x.distance)
                    matches.append((i, j, match))
                    print(f"Matches between Video {i + 1} and Video {j + 1}: {len(match)} matches found")

        # Calculate offsets based on the best matches
        for (i, j, match) in matches:
            if len(match) > 0:
                frame_diffs = []
                for m in match:
                    # Calculate the difference in frame index based on the matched keypoints
                    frame_diff = m.queryIdx - m.trainIdx
                    frame_diffs.append(frame_diff)
                if len(frame_diffs) > 0:
                    median_frame_diff = int(np.median(frame_diffs))
                    offsets[i, j] = median_frame_diff
                    offsets[j, i] = -median_frame_diff
                    print(f"Offset between Video {i + 1} and Video {j + 1}: {median_frame_diff} frames")

            # Draw matches for visualization
            frame_matches = cv2.drawMatches(frames[i], keypoints[i], frames[j], keypoints[j], match[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            frame_matches_resized = resize_frame(frame_matches, scale=0.5)
            window_name = f"Matches between Video {i + 1} and Video {j + 1}"
            cv2.imshow(window_name, frame_matches_resized)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    for cap in caps:
        cap.release()
    cv2.destroyAllWindows()

    # Convert offsets to seconds
    time_offsets = offsets / fps_list[0]  # Assuming all videos have the same FPS
    return offsets, time_offsets

if __name__ == "__main__":
    video_paths = ["IC1-08nov-TGI-Hazel-Jennie-Spring.mp4", "IC5-TGI-08nov-Hazel-Jennie-Spring.mp4", "IC3-TGI-08nov-Hazel-Jennie-Spring.mp4"]
    frame_offsets, time_offsets = find_sync_point(video_paths)
    print("Frame Offsets:")
    print(frame_offsets)
    print("Time Offsets (in seconds):")
    print(time_offsets)

    df_frame_offsets = pd.DataFrame(frame_offsets, index=video_paths, columns=video_paths)
    df_time_offsets = pd.DataFrame(time_offsets, index=video_paths, columns=video_paths)
    print("Frame Offsets with video names:")
    print(df_frame_offsets)
    print("Time Offsets (in seconds) with video names:")
    print(df_time_offsets)


Video 1: 500 keypoints detected
Video 2: 500 keypoints detected
Video 3: 500 keypoints detected
Matches between Video 1 and Video 2: 145 matches found
Matches between Video 1 and Video 3: 136 matches found
Matches between Video 2 and Video 3: 126 matches found
Offset between Video 1 and Video 2: -32 frames
Offset between Video 1 and Video 3: -52 frames
Offset between Video 2 and Video 3: 1 frames
Video 1: 500 keypoints detected
Video 2: 500 keypoints detected
Video 3: 500 keypoints detected
Matches between Video 1 and Video 2: 152 matches found
Matches between Video 1 and Video 3: 135 matches found
Matches between Video 2 and Video 3: 131 matches found
Offset between Video 1 and Video 2: -4 frames
Offset between Video 1 and Video 3: -37 frames
Offset between Video 2 and Video 3: -25 frames
Video 1: 500 keypoints detected
Video 2: 500 keypoints detected
Video 3: 500 keypoints detected
Matches between Video 1 and Video 2: 135 matches found
Matches between Video 1 and Video 3: 131 matches

KeyboardInterrupt: 

In [1]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1100)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        features.append(orb_features.mean(axis=0))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None
            
            print(f"Computing DTW shift for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

def align_and_display_videos(video_paths, time_shift_frames):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            frames.append(frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        cv2.imshow('Synchronized Videos', combined_frame)
        
        if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            break
    
    for cap in caps:
        cap.release()
    cv2.destroyAllWindows()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames)


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 framesProcessed 1100 frames

Processed 1100 frames
Processed 1200 frames
Processed 1100 frames
Processed 1200 frames
Processed 1300 frames
Processed 1200 frames
Processed 1300 frames
Processed 1400 frames
Processed 1300 frames
Processed 1400 frames
Processed 1500 frames
Processed 1400 frames
Processed 1500 frames
Processed 1600 frames
Processed 1600 frames
Processed 1500 

KeyboardInterrupt: 

In [1]:
import cv2
import numpy as np

def extract_features(video_path):
    cap = cv2.VideoCapture(video_path)
    orb = cv2.ORB_create()
    features = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = orb.detectAndCompute(gray, None)
        features.append((keypoints, descriptors))

    cap.release()
    return features

def match_features(features1, features2):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = []

    for (kp1, des1), (kp2, des2) in zip(features1, features2):
        if des1 is not None and des2 is not None:
            match = bf.match(des1, des2)
            matches.append(match)

    return matches

def find_best_match(matches):
    best_match_frame = 0
    max_matches = 0

    for i, match in enumerate(matches):
        if len(match) > max_matches:
            max_matches = len(match)
            best_match_frame = i

    return best_match_frame

# Extraire les caractéristiques des vidéos
features_video1 = extract_features(r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4")
features_video2 = extract_features(r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4")
features_video3 = extract_features(r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4")

# Correspondre les caractéristiques entre les vidéos
matches_12 = match_features(features_video1, features_video2)
matches_13 = match_features(features_video1, features_video3)

# Trouver les meilleurs correspondances et déterminer les décalages
best_frame_12 = find_best_match(matches_12)
best_frame_13 = find_best_match(matches_13)

delay_12 = best_frame_12
delay_13 = best_frame_13

print(f"Delay between video 1 and video 2: {delay_12} frames")
print(f"Delay between video 1 and video 3: {delay_13} frames")

# Synchroniser et afficher les vidéos
cap1 = cv2.VideoCapture(r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4")
cap2 = cv2.VideoCapture(r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4")
cap3 = cv2.VideoCapture(r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4")

# Sauter les frames pour synchroniser les vidéos
cap2.set(cv2.CAP_PROP_POS_FRAMES, delay_12)
cap3.set(cv2.CAP_PROP_POS_FRAMES, delay_13)

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    ret3, frame3 = cap3.read()

    if not ret1 or not ret2 or not ret3:
        break

    # Redimensionner les frames pour les afficher côte à côte
    height = min(frame1.shape[0], frame2.shape[0], frame3.shape[0])
    width = min(frame1.shape[1], frame2.shape[1], frame3.shape[1])

    frame1_resized = cv2.resize(frame1, (width, height))
    frame2_resized = cv2.resize(frame2, (width, height))
    frame3_resized = cv2.resize(frame3, (width, height))

    # Combiner les frames
    combined_frame = np.hstack((frame1_resized, frame2_resized, frame3_resized))
    cv2.imshow('Synchronized Videos', combined_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap1.release()
cap2.release()
cap3.release()
cv2.destroyAllWindows()


Delay between video 1 and video 2: 352 frames
Delay between video 1 and video 3: 68 frames


KeyboardInterrupt: 

In [5]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=2000)  # Augmenter le nombre de points clés
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None  # Retourner trois valeurs

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
            
            shifts = [p1 - p2 for p1, p2 in path]
            if len(shifts) == 0:
                print(f"Error: DTW returned empty path for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            
            shifts = np.array(shifts).reshape(-1, 1)
            indices = np.arange(len(shifts)).reshape(-1, 1)
            ransac = RANSACRegressor()
            ransac.fit(indices, shifts)
            inlier_mask = ransac.inlier_mask_
            
            median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths  # Retourner trois valeurs

def align_and_display_videos(video_paths, time_shift_frames):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            frames.append(frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        cv2.imshow('Synchronized Videos', combined_frame)
        
        if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            break
    
    for cap in caps:
        cap.release()
    cv2.destroyAllWindows()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames)

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 34585536 bytes in function 'cv::OutOfMemoryError'


In [1]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=2000)  # Augmenter le nombre de points clés
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_uint8 = np.nan_to_num(image * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None  # Retourner trois valeurs

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths  # Retourner trois valeurs

def align_and_display_videos(video_paths, time_shift_frames):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            frames.append(frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        cv2.imshow('Synchronized Videos', combined_frame)
        
        if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            break
    
    for cap in caps:
        cap.release()
    cv2.destroyAllWindows()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames)


error: bad allocation

In [1]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)  # Augmenter légèrement le nombre de points clés
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None  # Retourner trois valeurs

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths  # Retourner trois valeurs

def align_and_display_videos(video_paths, time_shift_frames):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            frames.append(frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        cv2.imshow('Synchronized Videos', combined_frame)
        
        if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            break
    
    for cap in caps:
        cap.release()
    cv2.destroyAllWindows()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames)


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1000 frames
Processed 1100 frames
Processed 1200 frames
Processed 1100 frames
Processed 1200 frames
Processed 1300 frames
Processed 1200 frames
Processed 1300 frames
Processed 1400 frames
Processed 1300 frames
Processed 1400 frames
Processed 1500 frames
Processed 1400 frames
Processed 1500 frames
Processed 1600 frames
Processed 1500 frames
Processed 1600 

In [4]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break
        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")
    
    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None  # Retourner trois valeurs

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths  # Retourner trois valeurs

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1200 frames
Processed 1300 frames
Processed 1300 frames
Processed 1300 frames
Processed 1400 frames
Processed 1400 frames
Processed 1400 frames
Processed 1500 frames
Processed 1500 frames
Processed 1500 frames
Processed 1600 frames
Processed 1600 

In [ ]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return np.array([])

    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None  # Retourner trois valeurs

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths  # Retourner trois valeurs

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")


In [7]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return np.array([])

    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None  # Retourner trois valeurs

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None  # Retourner trois valeurs
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths  # Retourner trois valeurs

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1200 frames
Processed 1300 frames
Processed 1300 frames
Processed 1300 frames
Processed 1400 frames
Processed 1400 frames
Processed 1400 frames
Processed 1500 frames
Processed 1500 frames
Processed 1500 frames
Processed 1600 frames
Processed 1600 

In [1]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return np.array([])

    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")

Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1300 frames
Processed 1200 frames
Processed 1300 frames
Processed 1400 frames
Processed 1300 frames
Processed 1400 frames
Processed 1500 frames
Processed 1400 frames
Processed 1500 frames
Processed 1600 frames
Processed 1500 frames
Processed 1600 

KeyboardInterrupt: 

In [2]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return np.array([])

    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, video_paths

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, max_shift)
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")

Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1100 frames
Processed 1200 frames
Processed 1300 frames
Processed 1200 frames
Processed 1300 frames
Processed 1400 frames
Processed 1400 frames
Processed 1300 frames
Processed 1500 frames
Processed 1500 frames
Processed 1400 frames
Processed 1600 frames
Processed 1600 frames
Processed 1500 

: 

In [2]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from scipy.stats import wasserstein_distance
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return np.array([])

    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def compute_mean_feature_repositioning(features1, features2):
    if len(features1) != len(features2):
        raise ValueError("Feature sets must have the same length for MFR computation")
    return np.mean([np.linalg.norm(f1 - f2) for f1, f2 in zip(features1, features2)])

def compute_emd(features1, features2):
    return wasserstein_distance(features1, features2)

def compute_mean_temporal_error(shifts):
    return np.mean(np.abs(shifts))

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None, None, None, None  # Return six values

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    mfr_values = np.zeros((num_videos, num_videos))
    emd_values = np.zeros((num_videos, num_videos))
    mean_temporal_errors = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None, None, None, None  # Return six values
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None, None, None, None  # Return six values
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift

            print(f"Computing MFR for video {i} and video {j}...")
            mfr_values[i, j] = compute_mean_feature_repositioning(features_list[i], features_list[j])

            print(f"Computing EMD for video {i} and video {j}...")
            emd_values[i, j] = compute_emd(features_list[i].flatten(), features_list[j].flatten())

            print(f"Computing mean temporal error for video {i} and video {j}...")
            mean_temporal_errors[i, j] = compute_mean_temporal_error(time_shift_frames[i, j])
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, mfr_values, emd_values, mean_temporal_errors, video_paths  # Return six values

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    # Synchronize the videos by aligning them using the computed shifts
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, mfr_values, emd_values, mean_temporal_errors, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    df_mfr = pd.DataFrame(mfr_values, index=video_names, columns=video_names)
    df_emd = pd.DataFrame(emd_values, index=video_names, columns=video_names)
    df_mte = pd.DataFrame(mean_temporal_errors, index=video_names, columns=video_names)

    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    print("Mean Feature Repositioning (MFR) with video names:")
    print(df_mfr)
    print("Earth Mover's Distance (EMD) with video names:")
    print(df_emd)
    print("Mean Temporal Error with video names:")
    print(df_mte)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1300 frames
Processed 1200 frames
Processed 1300 frames
Processed 1400 frames
Processed 1400 frames
Processed 1300 frames
Processed 1500 frames
Processed 1500 frames
Processed 1400 frames
Processed 1600 frames
Processed 1600 frames
Processed 1500 

ValueError: Feature sets must have the same length for MFR computation

In [3]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from scipy.stats import wasserstein_distance
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return np.array([])

    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def compute_mean_feature_repositioning(features1, features2):
    min_length = min(len(features1), len(features2))
    features1 = features1[:min_length]
    features2 = features2[:min_length]
    return np.mean([np.linalg.norm(f1 - f2) for f1, f2 in zip(features1, features2)])

def compute_emd(features1, features2):
    return wasserstein_distance(features1, features2)

def compute_mean_temporal_error(shifts):
    return np.mean(np.abs(shifts))

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None, None, None, None  # Return six values

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    mfr_values = np.zeros((num_videos, num_videos))
    emd_values = np.zeros((num_videos, num_videos))
    mean_temporal_errors = np.zeros((num_videos, num_videos))
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None, None, None, None  # Return six values
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None, None, None, None  # Return six values
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift

            print(f"Computing MFR for video {i} and video {j}...")
            mfr_values[i, j] = compute_mean_feature_repositioning(features_list[i], features_list[j])

            print(f"Computing EMD for video {i} and video {j}...")
            emd_values[i, j] = compute_emd(features_list[i].flatten(), features_list[j].flatten())

            print(f"Computing mean temporal error for video {i} and video {j}...")
            mean_temporal_errors[i, j] = compute_mean_temporal_error(time_shift_frames[i, j])
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    return time_shift_frames, time_shift_seconds, mfr_values, emd_values, mean_temporal_errors, video_paths  # Return six values

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    # Synchronize the videos by aligning them using the computed shifts
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, mfr_values, emd_values, mean_temporal_errors, video_names = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)

    df_frames = pd.DataFrame(time_shift_frames, index=video_names, columns=video_names)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_names, columns=video_names)
    df_mfr = pd.DataFrame(mfr_values, index=video_names, columns=video_names)
    df_emd = pd.DataFrame(emd_values, index=video_names, columns=video_names)
    df_mte = pd.DataFrame(mean_temporal_errors, index=video_names, columns=video_names)

    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    print("Mean Feature Repositioning (MFR) with video names:")
    print(df_mfr)
    print("Earth Mover's Distance (EMD) with video names:")
    print(df_emd)
    print("Mean Temporal Error with video names:")
    print(df_mte)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1200 frames
Processed 1300 frames
Processed 1300 frames
Processed 1400 frames
Processed 1300 frames
Processed 1400 frames
Processed 1500 frames
Processed 1400 frames
Processed 1500 frames
Processed 1600 frames
Processed 1500 frames
Processed 1600 

In [9]:
print("Mean Feature Repositioning (MFR) with video names:")
print(df_mfr)
print("Earth Mover's Distance (EMD) with video names:")
print(df_emd)


Mean Feature Repositioning (MFR) with video names:
                                       IC1-08nov-TGI-Hazel-Jennie-Spring.mp4  \
IC1-08nov-TGI-Hazel-Jennie-Spring.mp4                                    0.0   
IC3-TGI-08nov-Hazel-Jennie-Spring.mp4                                    0.0   
IC5-TGI-08nov-Hazel-Jennie-Spring.mp4                                    0.0   

                                       IC3-TGI-08nov-Hazel-Jennie-Spring.mp4  \
IC1-08nov-TGI-Hazel-Jennie-Spring.mp4                               52.22173   
IC3-TGI-08nov-Hazel-Jennie-Spring.mp4                                0.00000   
IC5-TGI-08nov-Hazel-Jennie-Spring.mp4                                0.00000   

                                       IC5-TGI-08nov-Hazel-Jennie-Spring.mp4  
IC1-08nov-TGI-Hazel-Jennie-Spring.mp4                              35.564293  
IC3-TGI-08nov-Hazel-Jennie-Spring.mp4                              71.347282  
IC5-TGI-08nov-Hazel-Jennie-Spring.mp4                               0.

In [11]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from scipy.stats import wasserstein_distance
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return np.array([])

    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def compute_mean_feature_repositioning(features1, features2):
    min_length = min(len(features1), len(features2))
    features1 = features1[:min_length]
    features2 = features2[:min_length]
    return np.mean([np.linalg.norm(f1 - f2) for f1, f2 in zip(features1, features2)])

def compute_emd(features1, features2):
    return wasserstein_distance(features1, features2)

def compute_mean_temporal_error(shifts):
    return np.mean(np.abs(shifts))

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None, None  # Return four values

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    emd_values = []
    mean_temporal_errors = []
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None, None  # Return four values
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None, None  # Return four values
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift

            print(f"Computing EMD for video {i} and video {j}...")
            emd_value = compute_emd(features_list[i].flatten(), features_list[j].flatten())
            emd_values.append(emd_value)

            print(f"Computing mean temporal error for video {i} and video {j}...")
            mean_temporal_error = compute_mean_temporal_error(time_shift_frames[i, j])
            mean_temporal_errors.append(mean_temporal_error)
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    mean_emd = np.mean(emd_values)
    mean_mte = np.mean(mean_temporal_errors)
    
    return time_shift_frames, time_shift_seconds, mean_emd, mean_mte  # Return four values

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    # Synchronize the videos by aligning them using the computed shifts
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, mean_emd, mean_mte = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)
    print(f"Mean Earth Mover's Distance (EMD): {mean_emd}")
    print(f"Mean Temporal Error (MTE): {mean_mte}")

    df_frames = pd.DataFrame(time_shift_frames, index=video_paths, columns=video_paths)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_paths, columns=video_paths)

    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1100 frames
Processed 1300 frames
Processed 1300 frames
Processed 1200 frames
Processed 1400 frames
Processed 1400 frames
Processed 1300 frames
Processed 1500 frames
Processed 1500 frames
Processed 1400 frames
Processed 1600 frames
Processed 1600 frames
Processed 1500 

In [12]:
import numpy as np
import cv2
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from scipy.stats import wasserstein_distance
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return np.array([])

    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def compute_mean_feature_repositioning(features1, features2):
    min_length = min(len(features1), len(features2))
    features1 = features1[:min_length]
    features2 = features2[:min_length]
    return np.mean([np.linalg.norm(f1 - f2) for f1, f2 in zip(features1, features2)])

def compute_emd(features1, features2):
    return wasserstein_distance(features1, features2)

def compute_mean_temporal_error(shifts):
    return np.mean(np.abs(shifts))

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None, None  # Return four values

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    emd_values = []
    mean_temporal_errors = []
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            # Filter out None descriptors
            descriptors1 = [d for d in descriptors1 if d is not None]
            descriptors2 = [d for d in descriptors2 if d is not None]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None, None  # Return four values
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None, None  # Return four values
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift

            print(f"Computing EMD for video {i} and video {j}...")
            emd_value = compute_emd(features_list[i].flatten(), features_list[j].flatten())
            emd_values.append(emd_value)

            print(f"Computing mean temporal error for video {i} and video {j}...")
            mean_temporal_error = compute_mean_temporal_error(time_shift_frames[i, j])
            mean_temporal_errors.append(mean_temporal_error)
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    mean_emd = np.mean(emd_values)
    mean_mte = np.mean(mean_temporal_errors)
    
    return time_shift_frames, time_shift_seconds, mean_emd, mean_mte  # Return four values

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    # Synchronize the videos by aligning them using the computed shifts
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, mean_emd, mean_mte = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)
    print(f"Mean Earth Mover's Distance (EMD): {mean_emd}")
    print(f"Mean Temporal Error (MTE): {mean_mte}")

    df_frames = pd.DataFrame(time_shift_frames, index=video_paths, columns=video_paths)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_paths, columns=video_paths)

    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1000 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1100 frames
Processed 1300 frames
Processed 1300 frames
Processed 1200 frames
Processed 1400 frames
Processed 1400 frames
Processed 1300 frames
Processed 1500 frames
Processed 1500 frames
Processed 1400 frames
Processed 1600 frames
Processed 1600 frames
Processed 1500 

In [13]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from scipy.stats import wasserstein_distance
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return np.array([])

    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def compute_mean_feature_repositioning(features1, features2):
    min_length = min(len(features1), len(features2))
    features1 = features1[:min_length]
    features2 = features2[:min_length]
    return np.mean([np.linalg.norm(f1 - f2) for f1, f2 in zip(features1, features2)])

def compute_emd(features1, features2):
    return wasserstein_distance(features1, features2)

def compute_mean_temporal_error(shifts):
    return np.mean(np.abs(shifts))

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None, None  # Return four values

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    emd_values = []
    mean_temporal_errors = []
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None, None  # Return four values
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None, None  # Return four values
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift

            print(f"Computing EMD for video {i} and video {j}...")
            emd_value = compute_emd(features_list[i].flatten(), features_list[j].flatten())
            emd_values.append(emd_value)

            print(f"Computing mean temporal error for video {i} and video {j}...")
            mean_temporal_error = compute_mean_temporal_error(time_shift_frames[i, j])
            mean_temporal_errors.append(mean_temporal_error)
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    mean_emd = np.mean(emd_values)
    mean_mte = np.mean(mean_temporal_errors)
    
    return time_shift_frames, time_shift_seconds, mean_emd, mean_mte  # Return four values

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    max_shift = int(np.max(np.abs(time_shift_frames)))
    
    # Synchronize the videos by aligning them using the computed shifts
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, mean_emd, mean_mte = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)
    print(f"Mean Earth Mover's Distance (EMD): {mean_emd}")
    print(f"Mean Temporal Error (MTE): {mean_mte}")

    df_frames = pd.DataFrame(time_shift_frames, index=video_paths, columns=video_paths)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_paths, columns=video_paths)

    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1200 frames
Processed 1300 frames
Processed 1300 frames
Processed 1300 frames
Processed 1400 frames
Processed 1400 frames
Processed 1400 frames
Processed 1500 frames
Processed 1500 frames
Processed 1500 frames
Processed 1600 frames
Processed 1600 

In [14]:
import numpy as np
import cv2
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from sklearn.linear_model import RANSACRegressor
from scipy.stats import wasserstein_distance
from concurrent.futures import ThreadPoolExecutor

def extract_features(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create(nfeatures=1000)
    keypoints, descriptors = orb.detectAndCompute(gray_frame, None)
    return descriptors if descriptors is not None else np.array([])

def extract_hog_descriptor(image):
    hog = cv2.HOGDescriptor()
    image_resized = cv2.resize(image, (64, 128))
    image_uint8 = np.nan_to_num(image_resized * 255).astype('uint8')
    h = hog.compute(image_uint8)
    return h.flatten()

def compute_ssm(features):
    n = len(features)
    ssm = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            ssm[i, j] = np.linalg.norm(features[i] - features[j])
    return ssm

def extract_log_polar_patch(ssm, i, window_size=10):
    h, w = ssm.shape
    center = (i, i)
    log_polar_patch = cv2.warpPolar(ssm, (window_size, window_size), center, 
                                    maxRadius=min(center), flags=cv2.WARP_POLAR_LOG)
    return log_polar_patch

def extract_ssm_descriptor(ssm, i, window_size=10):
    log_polar_patch = extract_log_polar_patch(ssm, i, window_size)
    patch_resized = cv2.resize(log_polar_patch, (64, 128))
    descriptor = extract_hog_descriptor(patch_resized)
    return descriptor

def process_video(video_path, sample_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return np.array([])

    features = []
    frame_count = 0
    
    while True:
        for _ in range(sample_rate):
            ret, frame = cap.read()
            if not ret:
                break
        if not ret:
            break

        orb_features = extract_features(frame)
        if orb_features.size == 0:
            continue
        hog_features = extract_hog_descriptor(frame)
        if hog_features.size == 0:
            continue
        combined_features = np.concatenate([orb_features.mean(axis=0), hog_features])
        features.append(combined_features)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

    cap.release()
    return np.array(features)

def process_videos_concurrently(video_paths, sample_rate=1):
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_video, video_path, sample_rate) for video_path in video_paths]
        features_list = [future.result() for future in futures]
    return features_list

def compute_dtw_shift(descriptors1, descriptors2, sample_rate):
    _, path = fastdtw(descriptors1, descriptors2, dist=euclidean)
    shifts = [p1 - p2 for p1, p2 in path]
    if len(shifts) == 0:
        print(f"Error: DTW returned empty path")
        return None
    shifts = np.array(shifts).reshape(-1, 1)
    indices = np.arange(len(shifts)).reshape(-1, 1)
    ransac = RANSACRegressor()
    ransac.fit(indices, shifts)
    inlier_mask = ransac.inlier_mask_
    if np.sum(inlier_mask) == 0:
        print("Error: No inliers found by RANSAC")
        return None
    median_shift = int(np.median(shifts[inlier_mask]) * sample_rate)
    return median_shift

def compute_emd(features1, features2):
    return wasserstein_distance(features1, features2)

def compute_mean_temporal_error(shifts):
    return np.mean(np.abs(shifts))

def synchronize_videos(video_paths, sample_rate=1):
    features_list = process_videos_concurrently(video_paths, sample_rate)
    
    if any(len(features) == 0 for features in features_list):
        print(f"Error: No features extracted from one or more videos")
        return None, None, None, None

    num_videos = len(video_paths)
    time_shift_frames = np.zeros((num_videos, num_videos))
    time_shift_seconds = np.zeros((num_videos, num_videos))
    emd_values = []
    mean_temporal_errors = []
    
    for i in range(num_videos):
        for j in range(i + 1, num_videos):
            print(f"Computing SSMs for video {i} and video {j}...")
            ssm1 = compute_ssm(features_list[i])
            ssm2 = compute_ssm(features_list[j])
            
            print(f"Extracting SSM descriptors for video {i} and video {j}...")
            descriptors1 = [extract_ssm_descriptor(ssm1, k) for k in range(len(features_list[i]))]
            descriptors2 = [extract_ssm_descriptor(ssm2, k) for k in range(len(features_list[j]))]
            
            if len(descriptors1) == 0 or len(descriptors2) == 0:
                print(f"Error: No valid descriptors extracted for video {i} and video {j}")
                return None, None, None, None
            
            print(f"Performing DTW with fastdtw for video {i} and video {j}...")
            median_shift = compute_dtw_shift(descriptors1, descriptors2, sample_rate)
            if median_shift is None:
                print(f"Error: Failed to compute DTW shift for video {i} and video {j}")
                return None, None, None, None
            print(f"Median shift between video {i} and video {j}: {median_shift} frames")
            time_shift_frames[i, j] = median_shift
            time_shift_frames[j, i] = -median_shift

            print(f"Computing EMD for video {i} and video {j}...")
            emd_value = compute_emd(features_list[i].flatten(), features_list[j].flatten())
            emd_values.append(emd_value)

            print(f"Computing mean temporal error for video {i} and video {j}...")
            mean_temporal_error = compute_mean_temporal_error(time_shift_frames[i, j])
            mean_temporal_errors.append(mean_temporal_error)
    
    cap = cv2.VideoCapture(video_paths[0])
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    
    time_shift_seconds = time_shift_frames / fps
    
    mean_emd = np.mean(emd_values)
    mean_mte = np.mean(mean_temporal_errors)
    
    return time_shift_frames, time_shift_seconds, mean_emd, mean_mte

def align_and_display_videos(video_paths, time_shift_frames, output_path="output.mp4"):
    caps = [cv2.VideoCapture(video_path) for video_path in video_paths]
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    frame_width = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    out_width = frame_width * len(caps)
    out_height = frame_height
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (out_width, out_height))
    
    while True:
        frames = []
        for i, cap in enumerate(caps):
            ret, frame = cap.read()
            if not ret:
                break
            shift = int(time_shift_frames[0, i])
            if shift > 0:
                for _ in range(shift):
                    cap.read()
            elif shift < 0:
                for _ in range(-shift):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 1)
            if frame is not None:
                resized_frame = cv2.resize(frame, (frame_width, frame_height))
                frames.append(resized_frame)
        
        if len(frames) != len(caps):
            break
        
        combined_frame = np.hstack(frames)
        out.write(combined_frame)
    
    for cap in caps:
        cap.release()
    out.release()

# Application
video_paths = [
    r"IC1-08nov-TGI-Hazel-Jennie-Spring.mp4",
    r"IC3-TGI-08nov-Hazel-Jennie-Spring.mp4",
    r"IC5-TGI-08nov-Hazel-Jennie-Spring.mp4"
]
time_shift_frames, time_shift_seconds, mean_emd, mean_mte = synchronize_videos(video_paths, sample_rate=1)
if time_shift_frames is not None and time_shift_seconds is not None:
    print("Estimated Time Shifts (in frames):")
    print(time_shift_frames)
    print("Estimated Time Shifts (in seconds):")
    print(time_shift_seconds)
    print(f"Mean Earth Mover's Distance (EMD): {mean_emd}")
    print(f"Mean Temporal Error (MTE): {mean_mte}")

    df_frames = pd.DataFrame(time_shift_frames, index=video_paths, columns=video_paths)
    df_seconds = pd.DataFrame(time_shift_seconds, index=video_paths, columns=video_paths)

    print("Time Shifts (in frames) with video names:")
    print(df_frames)
    print("Time Shifts (in seconds) with video names:")
    print(df_seconds)
    
    align_and_display_videos(video_paths, time_shift_frames, output_path="synchronized_output.mp4")


Processed 100 frames
Processed 100 frames
Processed 100 frames
Processed 200 frames
Processed 200 frames
Processed 200 frames
Processed 300 frames
Processed 300 frames
Processed 300 frames
Processed 400 frames
Processed 400 frames
Processed 400 frames
Processed 500 frames
Processed 500 frames
Processed 500 frames
Processed 600 frames
Processed 600 frames
Processed 600 frames
Processed 700 frames
Processed 700 frames
Processed 700 frames
Processed 800 frames
Processed 800 frames
Processed 800 frames
Processed 900 frames
Processed 900 frames
Processed 900 frames
Processed 1000 frames
Processed 1000 frames
Processed 1000 frames
Processed 1100 frames
Processed 1100 frames
Processed 1100 frames
Processed 1200 frames
Processed 1200 frames
Processed 1200 frames
Processed 1300 frames
Processed 1300 frames
Processed 1300 frames
Processed 1400 frames
Processed 1400 frames
Processed 1400 frames
Processed 1500 frames
Processed 1500 frames
Processed 1500 frames
Processed 1600 frames
Processed 1600 

In [2]:
import cv2
import numpy as np

# Fonction pour créer un masque basé sur un seuil (peut être ajusté pour une meilleure segmentation)
def create_mask(frame, threshold_value=200):
    _, mask = cv2.threshold(frame, threshold_value, 255, cv2.THRESH_BINARY)
    mask = cv2.bitwise_not(mask)
    return mask

# Lire la vidéo
cap = cv2.VideoCapture('IC1-08nov-TGI-Hazel-Jennie-Spring.mp4')

# Initialiser ORB
orb = cv2.ORB_create()

# Initialiser BFMatcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Lire la première frame
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_mask = create_mask(prev_gray)
prev_kp, prev_des = orb.detectAndCompute(prev_gray, prev_mask)

# Définir la taille de la fenêtre d'affichage
window_width = 800
window_height = 600

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mask = create_mask(gray)

    # Détecter les points clés et calculer les descripteurs
    kp, des = orb.detectAndCompute(gray, mask)

    if prev_des is not None and des is not None:
        # Matcher les descripteurs
        matches = bf.match(prev_des, des)
        matches = sorted(matches, key=lambda x: x.distance)

        # Dessiner les correspondances
        frame_matches = cv2.drawMatches(prev_frame, prev_kp, frame, kp, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # Redimensionner l'image des correspondances pour l'affichage
        frame_matches_resized = cv2.resize(frame_matches, (window_width, window_height))

        # Afficher les correspondances dans une fenêtre redimensionnée
        cv2.imshow('Matches', frame_matches_resized)

    # Mettre à jour les variables pour la prochaine frame
    prev_frame = frame
    prev_gray = gray
    prev_mask = mask
    prev_kp = kp
    prev_des = des

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [1]:
import cv2
import numpy as np

# Fonction pour créer un masque basé sur un seuil (peut être ajusté pour une meilleure segmentation)
def create_mask(frame, threshold_value=200):
    _, mask = cv2.threshold(frame, threshold_value, 255, cv2.THRESH_BINARY)
    mask = cv2.bitwise_not(mask)
    return mask

# Lire les vidéos
cap1 = cv2.VideoCapture('IC1-08nov-TGI-Hazel-Jennie-Spring.mp4')
cap2 = cv2.VideoCapture('IC3-TGI-08nov-Hazel-Jennie-Spring.mp4')
# Vérifier si les vidéos sont ouvertes correctement
if not cap1.isOpened() or not cap2.isOpened():
    print("Erreur: Impossible de lire une des vidéos.")
    exit()

# Initialiser ORB
orb = cv2.ORB_create()

# Initialiser BFMatcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Définir la taille de la fenêtre d'affichage
window_width = 800
window_height = 600

while cap1.isOpened() and cap2.isOpened():
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    if not ret1 or not ret2:
        break

    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    mask1 = create_mask(gray1)
    mask2 = create_mask(gray2)

    # Détecter les points clés et calculer les descripteurs
    kp1, des1 = orb.detectAndCompute(gray1, mask1)
    kp2, des2 = orb.detectAndCompute(gray2, mask2)

    if des1 is not None and des2 is not None:
        # Matcher les descripteurs
        matches = bf.match(des1, des2)
        matches = sorted(matches, key=lambda x: x.distance)

        # Dessiner les correspondances
        frame_matches = cv2.drawMatches(frame1, kp1, frame2, kp2, matches[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # Redimensionner l'image des correspondances pour l'affichage
        frame_matches_resized = cv2.resize(frame_matches, (window_width, window_height))

        # Afficher les correspondances dans une fenêtre redimensionnée
        cv2.imshow('Matches', frame_matches_resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap1.release()
cap2.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [5]:
import cv2
import numpy as np

# Fonction pour créer un masque basé sur un seuil (peut être ajusté pour une meilleure segmentation)
def create_mask(frame, threshold_value=200):
    _, mask = cv2.threshold(frame, threshold_value, 255, cv2.THRESH_BINARY)
    mask = cv2.bitwise_not(mask)
    return mask

# Lire les vidéos
cap1 = cv2.VideoCapture('IC1-08nov-TGI-Hazel-Jennie-Spring.mp4')
cap2 = cv2.VideoCapture('IC3-TGI-08nov-Hazel-Jennie-Spring.mp4')
cap3 = cv2.VideoCapture('IC5-TGI-08nov-Hazel-Jennie-Spring.mp4')


# Vérifier si les vidéos sont ouvertes correctement
if not cap1.isOpened() or not cap2.isOpened() or not cap3.isOpened():
    print("Erreur: Impossible de lire une des vidéos.")
    exit()

# Initialiser ORB
orb = cv2.ORB_create()

# Initialiser BFMatcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Définir la taille de la fenêtre d'affichage
window_width = 800
window_height = 600

while cap1.isOpened() and cap2.isOpened() and cap3.isOpened():
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    ret3, frame3 = cap3.read()
    if not ret1 or not ret2 or not ret3:
        break

    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    gray3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)
    mask1 = create_mask(gray1)
    mask2 = create_mask(gray2)
    mask3 = create_mask(gray3)

    # Détecter les points clés et calculer les descripteurs
    kp1, des1 = orb.detectAndCompute(gray1, mask1)
    kp2, des2 = orb.detectAndCompute(gray2, mask2)
    kp3, des3 = orb.detectAndCompute(gray3, mask3)

    if des1 is not None and des2 is not None and des3 is not None:
        # Matcher les descripteurs entre video1 et video2
        matches12 = bf.match(des1, des2)
        matches12 = sorted(matches12, key=lambda x: x.distance)

        # Matcher les descripteurs entre video2 et video3
        matches23 = bf.match(des2, des3)
        matches23 = sorted(matches23, key=lambda x: x.distance)

        # Dessiner les correspondances entre video1 et video2
        frame_matches12 = cv2.drawMatches(frame1, kp1, frame2, kp2, matches12[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # Dessiner les correspondances entre video2 et video3
        frame_matches23 = cv2.drawMatches(frame2, kp2, frame3, kp3, matches23[:10], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        # Redimensionner l'image des correspondances pour l'affichage
        frame_matches12_resized = cv2.resize(frame_matches12, (window_width, window_height))
        frame_matches23_resized = cv2.resize(frame_matches23, (window_width, window_height))

        # Afficher les correspondances dans des fenêtres redimensionnées
        cv2.imshow('Matches Video1-Video2', frame_matches12_resized)
        cv2.imshow('Matches Video2-Video3', frame_matches23_resized)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap1.release()
cap2.release()
cap3.release()
cv2.destroyAllWindows()


: 

In [4]:
import cv2
import numpy as np

# Fonction pour créer un masque basé sur un seuil (peut être ajusté pour une meilleure segmentation)
def create_mask(frame, threshold_value=200):
    _, mask = cv2.threshold(frame, threshold_value, 255, cv2.THRESH_BINARY)
    mask = cv2.bitwise_not(mask)
    return mask

# Lire les vidéos
cap1 = cv2.VideoCapture('IC1-08nov-TGI-Hazel-Jennie-Spring.mp4')
cap2 = cv2.VideoCapture('IC3-TGI-08nov-Hazel-Jennie-Spring.mp4')
cap3 = cv2.VideoCapture('IC5-TGI-08nov-Hazel-Jennie-Spring.mp4')

# Vérifier si les vidéos sont ouvertes correctement
if not cap1.isOpened() or not cap2.isOpened() or not cap3.isOpened():
    print("Erreur: Impossible de lire une des vidéos.")
    exit()

# Initialiser ORB
orb = cv2.ORB_create()

# Initialiser BFMatcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Variables pour stocker les décalages
offsets = {'video1-video2': [], 'video2-video3': []}

while cap1.isOpened() and cap2.isOpened() and cap3.isOpened():
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    ret3, frame3 = cap3.read()
    if not ret1 or not ret2 or not ret3:
        break

    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    gray3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)
    mask1 = create_mask(gray1)
    mask2 = create_mask(gray2)
    mask3 = create_mask(gray3)

    # Détecter les points clés et calculer les descripteurs
    kp1, des1 = orb.detectAndCompute(gray1, mask1)
    kp2, des2 = orb.detectAndCompute(gray2, mask2)
    kp3, des3 = orb.detectAndCompute(gray3, mask3)

    if des1 is not None and des2 is not None and des3 is not None:
        # Matcher les descripteurs entre video1 et video2
        matches12 = bf.match(des1, des2)
        matches12 = sorted(matches12, key=lambda x: x.distance)

        # Matcher les descripteurs entre video2 et video3
        matches23 = bf.match(des2, des3)
        matches23 = sorted(matches23, key=lambda x: x.distance)

        # Obtenir les points correspondants
        pts1 = np.float32([kp1[m.queryIdx].pt for m in matches12]).reshape(-1, 1, 2)
        pts2_1 = np.float32([kp2[m.trainIdx].pt for m in matches12]).reshape(-1, 1, 2)
        pts2_2 = np.float32([kp2[m.queryIdx].pt for m in matches23]).reshape(-1, 1, 2)
        pts3 = np.float32([kp3[m.trainIdx].pt for m in matches23]).reshape(-1, 1, 2)

        # Estimer la transformation entre video1 et video2
        H12, mask12 = cv2.findHomography(pts1, pts2_1, cv2.RANSAC, 5.0)

        # Estimer la transformation entre video2 et video3
        H23, mask23 = cv2.findHomography(pts2_2, pts3, cv2.RANSAC, 5.0)

        if H12 is not None and H23 is not None:
            # Calculer les décalages
            offset12 = np.mean(H12[0:2, 2])
            offset23 = np.mean(H23[0:2, 2])
            offsets['video1-video2'].append(offset12)
            offsets['video2-video3'].append(offset23)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap1.release()
cap2.release()
cap3.release()
cv2.destroyAllWindows()

# Calculer le décalage moyen
avg_offset12 = np.mean(offsets['video1-video2'])
avg_offset23 = np.mean(offsets['video2-video3'])

print(f"Décalage moyen entre video1 et video2: {avg_offset12}")
print(f"Décalage moyen entre video2 et video3: {avg_offset23}")


KeyboardInterrupt: 

In [5]:
import cv2
import numpy as np

def fast_detector(image, mask, threshold=50, n=9):
    """
    Détecteur de coins FAST avec masque pour ignorer certaines régions.
    """
    keypoints = []
    rows, cols = image.shape
    for y in range(n, rows - n):
        for x in range(n, cols - n):
            if mask[y, x] == 0:  # Ignorer les points masqués
                continue
            center = image[y, x]
            circle_pixels = [
                image[y - n, x], image[y - int(0.707 * n), x + int(0.707 * n)], 
                image[y, x + n], image[y + int(0.707 * n), x + int(0.707 * n)], 
                image[y + n, x], image[y + int(0.707 * n), x - int(0.707 * n)], 
                image[y, x - n], image[y - int(0.707 * n), x - int(0.707 * n)]
            ]
            brighter = [p > center + threshold for p in circle_pixels]
            darker = [p < center - threshold for p in circle_pixels]
            if sum(brighter) >= 3 or sum(darker) >= 3:
                keypoints.append((x, y))
    return keypoints

def brief_descriptor(image, keypoints, patch_size=31):
    """
    Descripteur BRIEF.
    """
    descriptors = []
    half_size = patch_size // 2
    for x, y in keypoints:
        if x < half_size or x >= image.shape[1] - half_size or y < half_size or y >= image.shape[0] - half_size:
            continue
        patch = image[y - half_size:y + half_size + 1, x - half_size:x + half_size + 1]
        descriptor = []
        for i in range(256):
            p1 = patch[np.random.randint(0, patch_size), np.random.randint(0, patch_size)]
            p2 = patch[np.random.randint(0, patch_size), np.random.randint(0, patch_size)]
            descriptor.append(1 if p1 < p2 else 0)
        descriptors.append(descriptor)
    return np.array(descriptors, dtype=np.uint8)

def orb_from_scratch(image, mask):
    """
    Fonction ORB principale avec masque.
    """
    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Détection des points clés avec FAST et masque
    keypoints = fast_detector(gray, mask)
    
    # Description des points clés avec BRIEF
    descriptors = brief_descriptor(gray, keypoints)
    
    return keypoints, descriptors

def create_dynamic_mask(frame, exclude_bottom_fraction=0.3):
    """
    Crée un masque dynamique pour ignorer le sol.
    """
    mask = np.ones(frame.shape[:2], dtype=np.uint8)
    height = frame.shape[0]
    exclude_start_row = int((1 - exclude_bottom_fraction) * height)
    mask[exclude_start_row:, :] = 0
    return mask

def process_video(video_path, exclude_bottom_fraction=0.3):
    """
    Traite une vidéo pour détecter les points clés, calculer les descripteurs, et afficher les résultats.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erreur: Impossible de lire la vidéo.")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Créer un masque dynamique pour ignorer le sol
        mask = create_dynamic_mask(frame, exclude_bottom_fraction)
        
        keypoints, descriptors = orb_from_scratch(frame, mask)

        # Affichage des points clés détectés
        for x, y in keypoints:
            cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)
        
        cv2.imshow('ORB from Scratch', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Chemin de la vidéo
video_path = 'IC1-08nov-TGI-Hazel-Jennie-Spring.mp4'

# Exécution de l'algorithme ORB from scratch sur la vidéo en excluant les 30% inférieurs
process_video(video_path, exclude_bottom_fraction=0.3)


KeyboardInterrupt: 

In [2]:
import cv2
import numpy as np

def fast_detector(image, mask, threshold=50, n=9):
    """
    Détecteur de coins FAST avec masque pour ignorer certaines régions.
    """
    keypoints = []
    rows, cols = image.shape
    for y in range(n, rows - n):
        for x in range(n, cols - n):
            if mask[y, x] == 0:  # Ignorer les points masqués
                continue
            center = image[y, x]
            circle_pixels = [
                image[y - n, x], image[y - int(0.707 * n), x + int(0.707 * n)], 
                image[y, x + n], image[y + int(0.707 * n), x + int(0.707 * n)], 
                image[y + n, x], image[y + int(0.707 * n), x - int(0.707 * n)], 
                image[y, x - n], image[y - int(0.707 * n), x - int(0.707 * n)]
            ]
            brighter = [p > center + threshold for p in circle_pixels]
            darker = [p < center - threshold for p in circle_pixels]
            if sum(brighter) >= 3 or sum(darker) >= 3:
                keypoints.append((x, y))
    return keypoints

def brief_descriptor(image, keypoints, patch_size=31):
    """
    Descripteur BRIEF.
    """
    descriptors = []
    half_size = patch_size // 2
    for x, y in keypoints:
        if x < half_size or x >= image.shape[1] - half_size or y < half_size or y >= image.shape[0] - half_size:
            continue
        patch = image[y - half_size:y + half_size + 1, x - half_size:x + half_size + 1]
        descriptor = []
        for i in range(256):
            p1 = patch[np.random.randint(0, patch_size), np.random.randint(0, patch_size)]
            p2 = patch[np.random.randint(0, patch_size), np.random.randint(0, patch_size)]
            descriptor.append(1 if p1 < p2 else 0)
        descriptors.append(descriptor)
    return np.array(descriptors, dtype=np.uint8)

def filter_keypoints_by_response(keypoints, response_threshold=30):
    """
    Filtrer les points clés en fonction de la réponse du détecteur.
    """
    filtered_keypoints = [kp for kp in keypoints if kp.response > response_threshold]
    return filtered_keypoints

def orb_from_scratch(image, mask, response_threshold=30):
    """
    Fonction ORB principale avec masque et filtrage des points clés pertinents.
    """
    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Détection des points clés avec FAST et masque
    fast = cv2.FastFeatureDetector_create()
    keypoints = fast.detect(gray, mask)
    
    # Filtrer les points clés en fonction de la réponse du détecteur
    keypoints = filter_keypoints_by_response(keypoints, response_threshold)
    
    # Description des points clés avec BRIEF
    descriptors = brief_descriptor(gray, [(int(kp.pt[0]), int(kp.pt[1])) for kp in keypoints])
    
    return keypoints, descriptors

def create_dynamic_mask(frame, exclude_bottom_fraction=0.3):
    """
    Crée un masque dynamique pour ignorer le sol.
    """
    mask = np.ones(frame.shape[:2], dtype=np.uint8)
    height = frame.shape[0]
    exclude_start_row = int((1 - exclude_bottom_fraction) * height)
    mask[exclude_start_row:, :] = 0
    return mask

def process_video(video_path, exclude_bottom_fraction=0.3, window_size=(800, 600)):
    """
    Traite une vidéo pour détecter les points clés, calculer les descripteurs, et afficher les résultats.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erreur: Impossible de lire la vidéo.")
        return

    # Créer la fenêtre d'affichage avec la taille spécifiée
    cv2.namedWindow('ORB from Scratch', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('ORB from Scratch', window_size[0], window_size[1])

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Créer un masque dynamique pour ignorer le sol
        mask = create_dynamic_mask(frame, exclude_bottom_fraction)
        
        keypoints, descriptors = orb_from_scratch(frame, mask)

        # Affichage des points clés détectés
        for kp in keypoints:
            x, y = int(kp.pt[0]), int(kp.pt[1])
            cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)
        
        cv2.imshow('ORB from Scratch', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Chemin de la vidéo
video_path = 'IC1-08nov-TGI-Hazel-Jennie-Spring.mp4'

# Exécution de l'algorithme ORB from scratch sur la vidéo en excluant les 30% inférieurs et avec une taille de fenêtre réduite
process_video(video_path, exclude_bottom_fraction=0.3, window_size=(1000, 500))


KeyboardInterrupt: 

In [3]:
import cv2
import numpy as np

def fast_detector(image, mask, threshold=50, n=9):
    """
    Détecteur de coins FAST avec masque pour ignorer certaines régions.
    """
    keypoints = []
    rows, cols = image.shape
    for y in range(n, rows - n):
        for x in range(n, cols - n):
            if mask[y, x] == 0:  # Ignorer les points masqués
                continue
            center = image[y, x]
            circle_pixels = [
                image[y - n, x], image[y - int(0.707 * n), x + int(0.707 * n)], 
                image[y, x + n], image[y + int(0.707 * n), x + int(0.707 * n)], 
                image[y + n, x], image[y + int(0.707 * n), x - int(0.707 * n)], 
                image[y, x - n], image[y - int(0.707 * n), x - int(0.707 * n)]
            ]
            brighter = [p > center + threshold for p in circle_pixels]
            darker = [p < center - threshold for p in circle_pixels]
            if sum(brighter) >= 3 or sum(darker) >= 3:
                keypoints.append((x, y))
    return keypoints

def brief_descriptor(image, keypoints, patch_size=31):
    """
    Descripteur BRIEF.
    """
    descriptors = []
    half_size = patch_size // 2
    for x, y in keypoints:
        if x < half_size or x >= image.shape[1] - half_size or y < half_size or y >= image.shape[0] - half_size:
            continue
        patch = image[y - half_size:y + half_size + 1, x - half_size:x + half_size + 1]
        descriptor = []
        for i in range(256):
            p1 = patch[np.random.randint(0, patch_size), np.random.randint(0, patch_size)]
            p2 = patch[np.random.randint(0, patch_size), np.random.randint(0, patch_size)]
            descriptor.append(1 if p1 < p2 else 0)
        descriptors.append(descriptor)
    return np.array(descriptors, dtype=np.uint8)

def filter_keypoints_by_response(keypoints, response_threshold=30):
    """
    Filtrer les points clés en fonction de la réponse du détecteur.
    """
    filtered_keypoints = [kp for kp in keypoints if kp.response > response_threshold]
    return filtered_keypoints

def orb_from_scratch(image, mask, response_threshold=30):
    """
    Fonction ORB principale avec masque et filtrage des points clés pertinents.
    """
    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Détection des points clés avec FAST et masque
    fast = cv2.FastFeatureDetector_create()
    keypoints = fast.detect(gray, mask)
    
    # Filtrer les points clés en fonction de la réponse du détecteur
    keypoints = filter_keypoints_by_response(keypoints, response_threshold)
    
    # Description des points clés avec BRIEF
    descriptors = brief_descriptor(gray, [(int(kp.pt[0]), int(kp.pt[1])) for kp in keypoints])
    
    return keypoints, descriptors

def create_static_mask(frame):
    """
    Crée un masque statique pour ignorer des régions spécifiques, comme le sol.
    """
    mask = np.ones(frame.shape[:2], dtype=np.uint8)
    # Exemple de masque pour ignorer une zone spécifique (le sol)
    # Adaptez les coordonnées selon votre besoin pour ignorer le sol
    height, width = frame.shape[:2]
    mask[int(0.7 * height):, :] = 0  # Masque la partie inférieure de l'image (le sol)
    return mask

def process_video(video_path, window_size=(800, 600)):
    """
    Traite une vidéo pour détecter les points clés, calculer les descripteurs, et afficher les résultats.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erreur: Impossible de lire la vidéo.")
        return

    # Créer la fenêtre d'affichage avec la taille spécifiée
    cv2.namedWindow('ORB from Scratch', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('ORB from Scratch', window_size[0], window_size[1])

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Créer un masque statique pour ignorer des régions spécifiques (le sol)
        mask = create_static_mask(frame)
        
        keypoints, descriptors = orb_from_scratch(frame, mask)

        # Affichage des points clés détectés
        for kp in keypoints:
            x, y = int(kp.pt[0]), int(kp.pt[1])
            cv2.circle(frame, (x, y), 3, (0, 255, 0), -1)
        
        cv2.imshow('ORB from Scratch', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Chemin de la vidéo
video_path = 'IC1-08nov-TGI-Hazel-Jennie-Spring.mp4'

# Exécution de l'algorithme ORB from scratch sur la vidéo avec une taille de fenêtre réduite
process_video(video_path, window_size=(800, 600))


KeyboardInterrupt: 